### 출력 파서

LLM이 생성한 출력 결과를 필요에 맞게 가공, 구조화

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [3]:
output_parser

StrOutputParser()

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1, # 창의성 (0.0 ~ 2.0)
    google_api_key=gemini_api_key
)


In [10]:
from langchain_core.prompts import PromptTemplate

template = "{language} 할 수 있어?"

prompt = PromptTemplate.from_template(template)

In [11]:
chain = prompt | model | output_parser # 프롬프트, 모델, 출력 파서 체인 구성

In [12]:
input = {"language": "한국말"}
chain.invoke(input)

'네, 한국말 할 수 있습니다. 무엇을 도와드릴까요?'

### 영어 회화 예제

In [ ]:
template = """
당신은 영어를 아주 쉽게 가르치는 친절한 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고해 주세요.

# 상황:
{question}

# FORMAT:
- 영어 회화:
- 한글 해석:
"""


In [14]:
prompt = PromptTemplate.from_template(template)

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1, # 창의성 (0.0 ~ 2.0)
    google_api_key=gemini_api_key
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser


In [16]:
result = chain.invoke({"question": "저는 식당에서 음식을 주문하고 싶어요."})


In [17]:
result

'안녕하세요! 식당에서 음식을 주문하는 상황, 아주 쉽고 친절하게 알려드릴게요. 걱정 마세요!\n\n---\n\n**# 상황: 저는 식당에서 음식을 주문하고 싶어요.**\n\n**[FORMAT]**\n\n- 영어 회화:\n    **You (손님):** Hello!\n    **Waiter (직원):** Hi there! Are you ready to order?\n    **You (손님):** Yes, I am. I\'d like the pasta, please.\n    **Waiter (직원):** Okay, the pasta. And anything to drink?\n    **You (손님):** Yes, and a Coke, please.\n    **Waiter (직원):** Alright, so that\'s one pasta and one Coke. Is that all for now?\n    **You (손님):** Yes, that\'s all. Thank you!\n    **Waiter (직원):** Great! It\'ll be right out.\n\n- 한글 해석:\n    **You (손님):** 안녕하세요!\n    **Waiter (직원):** 안녕하세요! 주문하시겠어요?\n    **You (손님):** 네, 할게요. 파스타로 부탁해요.\n    **Waiter (직원):** 네, 파스타요. 마실 것은요?\n    **You (손님):** 네, 콜라 한 잔 주세요.\n    **Waiter (직원):** 알겠습니다, 파스타 하나랑 콜라 하나요. 지금은 이게 다인가요?\n    **You (손님):** 네, 그게 다예요. 감사합니다!\n    **Waiter (직원):** 좋아요! 곧 가져다 드릴게요.\n\n---\n\n**✨ 선생님의 꿀팁! ✨**\n\n*   **"I\'d like ~"** (아이드 라이크 ~)는 "저는 ~을 원해요" 라는 아주 공손하고 자연스러운 표현이에요. "I want ~" 보다 훨씬 부드럽게 들

In [18]:
print(result)

안녕하세요! 식당에서 음식을 주문하는 상황, 아주 쉽고 친절하게 알려드릴게요. 걱정 마세요!

---

**# 상황: 저는 식당에서 음식을 주문하고 싶어요.**

**[FORMAT]**

- 영어 회화:
    **You (손님):** Hello!
    **Waiter (직원):** Hi there! Are you ready to order?
    **You (손님):** Yes, I am. I'd like the pasta, please.
    **Waiter (직원):** Okay, the pasta. And anything to drink?
    **You (손님):** Yes, and a Coke, please.
    **Waiter (직원):** Alright, so that's one pasta and one Coke. Is that all for now?
    **You (손님):** Yes, that's all. Thank you!
    **Waiter (직원):** Great! It'll be right out.

- 한글 해석:
    **You (손님):** 안녕하세요!
    **Waiter (직원):** 안녕하세요! 주문하시겠어요?
    **You (손님):** 네, 할게요. 파스타로 부탁해요.
    **Waiter (직원):** 네, 파스타요. 마실 것은요?
    **You (손님):** 네, 콜라 한 잔 주세요.
    **Waiter (직원):** 알겠습니다, 파스타 하나랑 콜라 하나요. 지금은 이게 다인가요?
    **You (손님):** 네, 그게 다예요. 감사합니다!
    **Waiter (직원):** 좋아요! 곧 가져다 드릴게요.

---

**✨ 선생님의 꿀팁! ✨**

*   **"I'd like ~"** (아이드 라이크 ~)는 "저는 ~을 원해요" 라는 아주 공손하고 자연스러운 표현이에요. "I want ~" 보다 훨씬 부드럽게 들린답니다!
*   음식 이름 뒤에 **"please"** (플리즈)를

### JsonOutputParser

In [20]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# 1. 원하는 출력 형식을 정의하는 Pydantic BaseModel 생성
#    이것은 LLM에게 어떤 키와 값이 필요한지 알려주는 역할을 합니다.
class EnglishLesson(BaseModel):
    english_conversation: str = Field(description="상황에 맞는 영어 회화")
    korean_translation: str = Field(description="영어 회화의 한글 해석")


In [21]:
# 2. JsonOutputParser 인스턴스 생성
#    Pydantic 모델을 파서에 연결하여 LLM이 정확한 JSON을 생성하도록 유도합니다.
parser = JsonOutputParser(pydantic_object=EnglishLesson)

In [28]:
# 3. 프롬프트 템플릿에 파싱 지시문 추가
#    parser.get_format_instructions()는 LLM에게 JSON 출력 형식을 정확히 지시하는 문자열을 반환합니다.
from langchain_core.prompts import PromptTemplate
template = """
당신은 영어를 아주 쉽게 가르치는 친절한 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고해 주세요.

# 상황:
{question}

# FORMAT:
{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [29]:
# 4. LLM 모델 및 체인 구성
prompt = PromptTemplate.from_template(template)

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1, # 창의성 (0.0 ~ 2.0)
    google_api_key=gemini_api_key
)

chain = prompt | model | parser

In [30]:
# 5. 체인 실행 및 결과 출력
#    응답이 JSON 객체로 파싱되어 출력됩니다.
user_question = "친구와 저녁 메뉴를 정하는 상황"
response = chain.invoke({"question": user_question})

KeyError: "Input to PromptTemplate is missing variables {'format_instructions'}.  Expected: ['format_instructions', 'question'] Received: ['question']\nNote: if you intended {format_instructions} to be part of the string and not a variable, please escape it with double curly braces like: '{{format_instructions}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "